# Parte 1 - Agrupaciones y Promedio

¿Como funciona?

1.1 Descarga el historico de Google sheets

1.2 Calcula D2S y D2S en pix de venta con las diferentes agrupaciones:

['SKU']

['Marca' , 'Modelo' , 'Year Group', 'transmision']

['Marca' , 'Modelo' , 'Year Group']

['gama' , 'Body Type', 'Price Group', 'Year Group']

['gama' , 'Body Type', 'Year Group']

['Body Type', 'Price Group', 'Year Group']

['Body Type', 'Price Group']

['Body Type']


1.3 Descarga el catalogo de Google Sheets

1.4 Cruce catálogo con las diiferentes agrupaciones
- Prioridad 1, 2 ,3, 4, 5... corresponden a las agrupaciones.


1.9 Carga a Google Sheets

Link: xxxxxxxx


Notas:
- Quedo sin requisito de data points. Se cambia en 1.2
- Calculo D2S en PIX de venta
- Days available son D2S pero tomados desde el available en la pag web
- Carros con publicación desde antes del 2022-06-22 no son tenidos en cuenta por falta de datos
- Para grupos de KM 7000 por año es avg. Quite el KM porque en el Catálogo no lo tenemos de esta forma. El calculo igual se puede hacer facil.


In [1]:
import pandas as pd
import numpy as np

import os
pathout = './data/'
if not os.path.exists(pathout):
    os.mkdir(pathout) 


### 1.1 Descargar historico desde Google Sheets

In [2]:
from __future__ import print_function
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
import json

In [3]:
# Set the scope and credentials
scope = ['https://www.googleapis.com/auth/spreadsheets']
creds = None
creds = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope)
# creds = Credentials.from_authorized_user_file('creds.json', scope)
# Authenticate with Google Sheets


client = gspread.authorize(creds)
SPREADSHEET_ID = 'xxxxxxxxx'
RANGE_NAME = "Vendidos!A1:AF2500"
service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                             range=RANGE_NAME).execute()

values = result.get('values', [])

df = pd.DataFrame(values[1:], columns=values[0])

#### Limpiar datos

In [4]:
df = df.replace(' ', np.nan)
df = df.replace('', np.nan)
df = df.dropna(subset=['Dominio'])
df.replace('', float('nan'), inplace=True)

column_names =['Elegible por logica de D2S vs Available' ,
               'Elegible por disponibilidad de datos',
               'Año',
               'PM Actual - LC',
               'Days Available',
               'D2S en PIX venta',
               'D2S',
               'PIX salida',
               'PX% Real']

for column in column_names:
    df[column] = df[column].replace('#N/A', np.nan)
    df[column] = df[column].astype(str)
    df[column] = df[column].replace('None', np.nan)
    df[column] = df[column].str.replace(',', '.')
    df[column] = df[column].astype(float)



df['Marca'] = df['Marca'].str.upper()
df['Modelo'] = df['Modelo'].str.upper()


### Vendidos

In [5]:

cars = df

cars['Vendido'] = cars['Vendido'].astype(float)

cars = cars.drop(cars[cars['Elegible por logica de D2S vs Available'] != 1].index)
cars = cars.drop(cars[cars['Elegible por disponibilidad de datos'] != 1].index)
cars = cars.drop(cars[cars['Vendido'] != 1].index)



# cars.head(1)
# cars.dtypes

### Todos

In [6]:

HistCars = df
HistCars = HistCars.drop(HistCars[HistCars['Elegible por disponibilidad de datos'] != 1].index)
HistCars = HistCars.drop(HistCars[HistCars['STATUS 2'] == '09.3-Suspendido'].index)
HistCars = HistCars.drop(HistCars[HistCars['STATUS 2'] == '99-Devolucion'].index)
HistCars = HistCars.drop(HistCars[HistCars['STATUS 2'] == '99-Baja'].index)


# HistCars.head(1)

### 1.2 Calcular el promedio de D2S y D2S en pix de venta en las diferentes agrupaciones

In [8]:
import pandas as pd

agrupacion1 = ['SKU']
agrupacion2 = ['Marca' , 'Modelo' , 'Year Group', 'transmision']
agrupacion3 = ['Marca' , 'Modelo' , 'Year Group']
agrupacion4 = ['Gama' , 'Body Type', 'Price Group', 'Year Group']
agrupacion5 = ['Gama' , 'Body Type', 'Year Group']
agrupacion7 = ['Body Type', 'Price Group', 'Year Group']
agrupacion8 = ['Body Type', 'Price Group']
agrupacion9 = ['Body Type' , 'Gama']
agrupacion10 = ['Body Type']


# Alternativas con KM
# agrupacion1 = ['SKU']
# agrupacion2 = ['Body Type', 'Price Group', 'Year Group', 'KmGroup']
# agrupacion3 = ['Body Type', 'Price Group', 'KmGroup']
# agrupacion4 = ['Body Type', 'KmGroup']
# agrupacion5 = ['KmGroup']


list_of_agrup = [agrupacion1, agrupacion2, agrupacion3, agrupacion4, agrupacion5,
                 agrupacion6, agrupacion7, agrupacion8, agrupacion9, agrupacion10]

dataframes = {} 

counter = 1
for i, agrupaciones in enumerate(list_of_agrup):
    # Vendidos
    a = cars.groupby(agrupaciones)['D2S en PIX venta'].mean().reset_index()

    b = cars.groupby(agrupaciones)['D2S en PIX venta'].count().reset_index()
    b.rename(columns={'D2S en PIX venta': 'Cars Sold'}, inplace=True)

    c = cars.groupby(agrupaciones)['D2S'].mean().reset_index()
    d = cars.groupby(agrupaciones)['Days Available'].mean().reset_index()

    p1 = cars.groupby(agrupaciones)['PX% Real'].mean().reset_index()
    p1.rename(columns={'PX% Real': 'PIX de venta'}, inplace=True)
    p2 = cars.groupby(agrupaciones)['PIX salida'].mean().reset_index()

    agrupacion_df = a.merge(d, how='inner', on=agrupaciones).fillna(7777777)
    agrupacion_df = agrupacion_df.merge(c, how='inner', on=agrupaciones).fillna(7777777)
    agrupacion_df = agrupacion_df.merge(b, how='inner', on=agrupaciones).fillna(7777777)
    agrupacion_df = agrupacion_df.merge(p1, how='inner', on=agrupaciones).fillna(7777777)
    agrupacion_df = agrupacion_df.merge(p2, how='inner', on=agrupaciones).fillna(7777777)

    # Stock
    o = HistCars.groupby(agrupaciones)['Dominio'].count().reset_index()
    o.rename(columns={'Dominio': 'Total Cars'}, inplace=True)

    agrupacion_df = agrupacion_df.merge(o, how='inner', on=agrupaciones).fillna(7777777)

    agrupacion_df['% Sold'] = agrupacion_df['Cars Sold'] / agrupacion_df['Total Cars']
    agrupacion_df['Adjusted D2S venta'] = agrupacion_df['D2S en PIX venta'] / agrupacion_df['% Sold']
    agrupacion_df['Prioridad'] = counter
    counter += 1 
    agrupacion_df.replace([np.inf, -np.inf], "no valido", inplace=True)

    #Drop if columna 777777
    agrupacion_df = agrupacion_df[~(agrupacion_df == 7777777).any(axis=1)]
    #Keep 3 or more data points
    # agrupacion_df = agrupacion_df[agrupacion_df['Cars Sold'] >= 2]




    df_name = f"agrupacion{counter}"
    dataframes[df_name] = agrupacion_df 



agrupacion1 = None
agrupacion2 = None
agrupacion3 = None
agrupacion4 = None
agrupacion5 = None
agrupacion6 = None
agrupacion7 = None
agrupacion8 = None
agrupacion9 = None
agrupacion10 = None


for i, df_name in enumerate(dataframes.keys()):
    var_name = f"agrupacion{i + 1}"
    globals()[var_name] = dataframes[df_name]





In [9]:
#Datapoints. Por ahora 3 para cada agrupación
agrupacion1 = agrupacion1[agrupacion1['Cars Sold'] >= 2]
agrupacion2 = agrupacion2[agrupacion2['Cars Sold'] >= 3]
agrupacion3 = agrupacion3[agrupacion3['Cars Sold'] >= 3]
agrupacion4 = agrupacion4[agrupacion4['Cars Sold'] >= 3]
agrupacion5 = agrupacion5[agrupacion5['Cars Sold'] >= 3]
agrupacion6 = agrupacion6[agrupacion6['Cars Sold'] >= 3]
agrupacion7 = agrupacion7[agrupacion7['Cars Sold'] >= 3]
agrupacion8 = agrupacion8[agrupacion8['Cars Sold'] >= 3]
agrupacion9 = agrupacion9[agrupacion9['Cars Sold'] >= 3]
agrupacion10 = agrupacion10[agrupacion10['Cars Sold'] >= 3000]

#Por ahora exluir solo bodytype

### 1.3 Descargar el catálogo

In [10]:

scope = ['https://www.googleapis.com/auth/spreadsheets']
creds = None
creds = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope)



client = gspread.authorize(creds)

SPREADSHEET_ID = 'XXXXXXXXXXX'
RANGE_NAME = "Catalogo!B1:Q11200"
service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                             range=RANGE_NAME).execute()

values = result.get('values', [])

catalogo = pd.DataFrame(values[1:], columns=values[0])


catalogo.rename(columns={'sku': 'SKU' , 'bodytype' : 'Body Type',
                         'aod_make':'Marca' ,'model':'Modelo' }, inplace=True)




catalogo = catalogo[['SKU', 'Marca', 'Modelo' , 'year' , 'version_short_desc' ,
             'Body Type', 'Price Group' , 'Year Group', 'transmision', 'Gama' ]] 

In [11]:
catalogo.head(2)

,SKU,Marca,Modelo,year,version_short_desc,Body Type,Price Group,Year Group,transmision,Gama
0,CO-J021-P-22-1008-1002,JAGUAR,F-PACE,2022,3.0 R-DYNAMIC S MHEV,WAGON,,2021-23,Automatic,Alta
1,CO-J021-P-21-1008-1002,JAGUAR,F-PACE,2021,3.0 R-DYNAMIC S MHEV,WAGON,,2021-23,Automatic,Alta


### 1.4 Cruce catálogo con las diiferentes agrupaciones

In [12]:
#Catálogo con las predicciones

#1 Merge catalogo con predicciones a nivel SKU con mas de 3 data points
cat_pred = catalogo.merge(agrupacion1, how='inner', on=['SKU']).fillna(7777777)
cat_pred

#2.1 (debo arreglar los numeros) Merge catalogo limpio con catalogo con las nuevas predicciones
catalogo12 = cat_pred.merge(catalogo, how='outer', on=['SKU', 'Marca',
                                          'Modelo',	'year',
                                          'version_short_desc',	'Body Type',
                                          'Price Group',	'Year Group', 'transmision', 'Gama']).fillna(7777777)
# keep 77777 (son los que no cruzaron)
catalogo12 = catalogo12[catalogo12['D2S'] == 7777777]

catalogo12 = catalogo12[['SKU', 'Marca', 'Modelo' , 'year' , 'version_short_desc' ,
             'Body Type', 'Price Group' , 'Year Group', 'transmision' , 'Gama']] 


cat_pred12 = catalogo12.merge(agrupacion2, how='inner', on=['Marca' , 'Modelo','Year Group' , 'transmision']).fillna(7777777)
cat_pred12


#2 Merge catalogo limpio con catalogo con las nuevas predicciones
catalogo2 = cat_pred12.merge(catalogo12, how='outer', on=['SKU', 'Marca',
                                          'Modelo',	'year',
                                          'version_short_desc',	'Body Type',
                                          'Price Group',	'Year Group', 'transmision', 'Gama']).fillna(7777777)
# keep 77777 (son los que no cruzaron)
catalogo2 = catalogo2[catalogo2['D2S'] == 7777777]

catalogo2 = catalogo2[['SKU', 'Marca', 'Modelo' , 'year' , 'version_short_desc' ,
             'Body Type', 'Price Group' , 'Year Group' , 'transmision', 'Gama']] 


cat_pred2 = catalogo2.merge(agrupacion3, how='inner', on=['Marca' , 'Modelo' , 'Year Group']).fillna(7777777)
cat_pred2

#3 # repetir 2
catalogo3 = cat_pred2.merge(catalogo2, how='outer', on=['SKU', 'Marca',
                                          'Modelo',	'year',
                                          'version_short_desc',	'Body Type',
                                          'Price Group',	'Year Group', 'transmision', 'Gama']).fillna(7777777)

catalogo3 = catalogo3[catalogo3['D2S'] == 7777777]

catalogo3 = catalogo3[['SKU', 'Marca', 'Modelo' , 'year' , 'version_short_desc' ,
             'Body Type', 'Price Group' , 'Year Group', 'transmision' , 'Gama']] 


cat_pred3 = catalogo3.merge(agrupacion4, how='inner', on=['Gama' , 'Body Type', 'Price Group', 'Year Group']).fillna(7777777)
cat_pred3


#4
catalogo4 = cat_pred3.merge(catalogo3, how='outer', on=['SKU', 'Marca',
                                          'Modelo',	'year',
                                          'version_short_desc',	'Body Type',
                                          'Price Group',	'Year Group', 'transmision', 'Gama']).fillna(7777777)

catalogo4 = catalogo4[catalogo4['D2S'] == 7777777]

catalogo4 = catalogo4[['SKU', 'Marca', 'Modelo' , 'year' , 'version_short_desc' ,
             'Body Type', 'Price Group' , 'Year Group', 'transmision', 'Gama']] 


cat_pred4 = catalogo4.merge(agrupacion5, how='inner', on=['Gama' , 'Body Type', 'Year Group']).fillna(7777777)
cat_pred4

#5
catalogo5 = cat_pred4.merge(catalogo4, how='outer', on=['SKU', 'Marca',
                                          'Modelo',	'year',
                                          'version_short_desc',	'Body Type',
                                          'Price Group',	'Year Group', 'transmision', 'Gama']).fillna(7777777)

catalogo5 = catalogo5[catalogo5['D2S'] == 7777777]

catalogo5 = catalogo5[['SKU', 'Marca', 'Modelo' , 'year' , 'version_short_desc' ,
             'Body Type', 'Price Group' , 'Year Group', 'transmision', 'Gama' ]] 


cat_pred5 = catalogo5.merge(agrupacion6, how='inner', on=['Marca' , 'Modelo', 'Year Group']).fillna(7777777)

#6
catalogo6 = cat_pred5.merge(catalogo5, how='outer', on=['SKU', 'Marca',
                                          'Modelo',	'year',
                                          'version_short_desc',	'Body Type',
                                          'Price Group',	'Year Group', 'transmision', 'Gama']).fillna(7777777)

catalogo6 = catalogo6[catalogo6['D2S'] == 7777777]

catalogo6 = catalogo6[['SKU', 'Marca', 'Modelo' , 'year' , 'version_short_desc' ,
             'Body Type', 'Price Group' , 'Year Group' , 'transmision', 'Gama']] 


cat_pred6 = catalogo6.merge(agrupacion7, how='inner', on=['Body Type', 'Price Group', 'Year Group']).fillna(7777777)

#7
catalogo7 = cat_pred6.merge(catalogo6, how='outer', on=['SKU', 'Marca',
                                          'Modelo',	'year',
                                          'version_short_desc',	'Body Type',
                                          'Price Group',	'Year Group', 'transmision', 'Gama']).fillna(7777777)

catalogo7 = catalogo7[catalogo7['D2S'] == 7777777]

catalogo7 = catalogo7[['SKU', 'Marca', 'Modelo' , 'year' , 'version_short_desc' ,
             'Body Type', 'Price Group' , 'Year Group' , 'transmision', 'Gama']] 


cat_pred7 = catalogo7.merge(agrupacion8, how='inner', on=['Body Type', 'Price Group']).fillna(7777777)

#8
catalogo8 = cat_pred7.merge(catalogo7, how='outer', on=['SKU', 'Marca',
                                          'Modelo',	'year',
                                          'version_short_desc',	'Body Type',
                                          'Price Group',	'Year Group', 'transmision', 'Gama']).fillna(7777777)

catalogo8 = catalogo8[catalogo8['D2S'] == 7777777]

catalogo8 = catalogo8[['SKU', 'Marca', 'Modelo' , 'year' , 'version_short_desc' ,
             'Body Type', 'Price Group' , 'Year Group', 'transmision' , 'Gama']] 


cat_pred8 = catalogo8.merge(agrupacion9, how='inner', on=['Body Type', 'Gama']).fillna(7777777)

#9
catalogo9 = cat_pred8.merge(catalogo8, how='outer', on=['SKU', 'Marca',
                                          'Modelo',	'year',
                                          'version_short_desc',	'Body Type',
                                          'Price Group',	'Year Group', 'transmision', 'Gama']).fillna(7777777)

catalogo9 = catalogo9[catalogo9['D2S'] == 7777777]

catalogo9 = catalogo9[['SKU', 'Marca', 'Modelo' , 'year' , 'version_short_desc' ,
             'Body Type', 'Price Group' , 'Year Group' , 'transmision', 'Gama']] 


cat_pred9 = catalogo9.merge(agrupacion10, how='inner', on=['Body Type']).fillna(7777777)

###
catalogo_final = pd.concat([cat_pred,cat_pred12 , cat_pred2, cat_pred3, cat_pred4, cat_pred5,
                            cat_pred6 , cat_pred7, cat_pred8, cat_pred9], ignore_index=True)


catalogo_final = catalogo_final.merge(catalogo, how='outer', on=['SKU', 'Marca',
                                          'Modelo',	'year',
                                          'version_short_desc',	'Body Type',
                                          'Price Group',	'Year Group', 'transmision', 'Gama']).fillna('Sin Pred')
catalogo_final = catalogo_final.drop_duplicates(subset='SKU')



### 1.9 Carga a Google Sheets

#### 1.9.1 Carga Catálogo

In [13]:
listoflist = catalogo_final.values.tolist()
columns = catalogo_final.columns.tolist()
listoflist = [columns] + catalogo_final.values.tolist()


RANGE_NAME_UPDATE = 'Catálogo Nuevo D2S!A1'


request = sheet.values().update(spreadsheetId='10X4sHCdtZePP-vz4_gMDUnyDTblKLLeQoblRzAdZQ-M',
                                 range=RANGE_NAME_UPDATE,
                                 valueInputOption='USER_ENTERED',
                                 body={"values":listoflist})

response = request.execute()




print('DataFrame cargado correctamente en Google Sheets.')

DataFrame cargado correctamente en Google Sheets.


#### 1.9.2 Carga Agrupaciones

In [14]:
# Agrupacion 1
listoflist = agrupacion1.values.tolist()
columns = agrupacion1.columns.tolist()
listoflist = [columns] + agrupacion1.values.tolist()


RANGE_NAME_UPDATE = 'SKU!A1'


request = sheet.values().update(spreadsheetId='10X4sHCdtZePP-vz4_gMDUnyDTblKLLeQoblRzAdZQ-M',
                                 range=RANGE_NAME_UPDATE,
                                 valueInputOption='USER_ENTERED',
                                 body={"values":listoflist})

response = request.execute()

# Agrupacion 2
listoflist = agrupacion2.values.tolist()
columns = agrupacion2.columns.tolist()
listoflist = [columns] + agrupacion2.values.tolist()


RANGE_NAME_UPDATE = 'Marca-Modelo-YrGroup!A1'


request = sheet.values().update(spreadsheetId='10X4sHCdtZePP-vz4_gMDUnyDTblKLLeQoblRzAdZQ-M',
                                 range=RANGE_NAME_UPDATE,
                                 valueInputOption='USER_ENTERED',
                                 body={"values":listoflist})

response = request.execute()

# Agrupacion 3
listoflist = agrupacion3.values.tolist()
columns = agrupacion3.columns.tolist()
listoflist = [columns] + agrupacion3.values.tolist()


RANGE_NAME_UPDATE = 'BodyType-PriceG-YearG!A1'


request = sheet.values().update(spreadsheetId='10X4sHCdtZePP-vz4_gMDUnyDTblKLLeQoblRzAdZQ-M',
                                 range=RANGE_NAME_UPDATE,
                                 valueInputOption='USER_ENTERED',
                                 body={"values":listoflist})

response = request.execute()

# Agrupacion 4
listoflist = agrupacion4.values.tolist()
columns = agrupacion4.columns.tolist()
listoflist = [columns] + agrupacion4.values.tolist()


RANGE_NAME_UPDATE = 'BodyType-PriceG!A1'


request = sheet.values().update(spreadsheetId='10X4sHCdtZePP-vz4_gMDUnyDTblKLLeQoblRzAdZQ-M',
                                 range=RANGE_NAME_UPDATE,
                                 valueInputOption='USER_ENTERED',
                                 body={"values":listoflist})

response = request.execute()

# Agrupacion 5
listoflist = agrupacion5.values.tolist()
columns = agrupacion5.columns.tolist()
listoflist = [columns] + agrupacion5.values.tolist()


RANGE_NAME_UPDATE = 'BodyType!A1'


request = sheet.values().update(spreadsheetId='10X4sHCdtZePP-vz4_gMDUnyDTblKLLeQoblRzAdZQ-M',
                                 range=RANGE_NAME_UPDATE,
                                 valueInputOption='USER_ENTERED',
                                 body={"values":listoflist})

response = request.execute()


print('Agrupaciones cargadas correctamente en Google Sheets.')

Agrupaciones cargadas correctamente en Google Sheets.
